# Discussing Coroutines **Context**

- **Coroutine context** and **dispatchers** are discussed.

In [1]:
// Importing some neccessary libs in jupyter notebook
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import java.util.concurrent.Executors
import kotlin.coroutines.CoroutineContext

## Coroutine `Context`

- A coroutine **context** in Kotlin is a set of elements that define the behavior of a coroutine. It holds information such as the **dispatcher** (which determines which *thread* or *threads* the coroutine runs on), a **Job** that *controls the lifecycle* of the coroutine, and other optional elements that can customize coroutine execution.

1. `Job`
    - A `Job` controls the lifecycle of the coroutine. It can be used to track and manage the coroutine's state (active, canceled, or completed).
    - The `Job` is part of the **context**, and every coroutine has an associated (dedicated or inherited) `Job`.
3. `CoroutineDispatcher`
    - `Dispatchers.Default`: Uses a shared background pool of threads and is optimized for CPU-intensive tasks.
    - `Dispatchers.IO`: Optimized for I/O-bound tasks such as reading from files or network operations.
    - `Dispatchers.Main`: Confines the execution to the main thread, often used for updating UI components.
    - `Dispatchers.Unconfined`: The coroutine starts on the current thread but can resume on different threads. It is less commonly used.
    - **Customised Dispatcher**: It is possible to create a customised dispatcher with a given number of threads.
5. `CoroutineName`
   - A `CoroutineName` is an optional element that gives a name to a coroutine for debugging purposes.
  
* To create a custom context, the structure is like:
  ```kotlin
  val context: CoroutineContext = Dispatchers.IO + Job() + CoroutineName("Name") 
  ```

In [2]:
// Launching a coroutine within a given context

// As we saw, 'runBlocking' can also accept a customised context like 'runBlocking(context) {}'
fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started \n `running on $coroutineContext`")

    /* 
    Let's create a customized context.
    To create a custom context, the structure is like [CoroutineDispatcher]+[Job]+[CoroutineName].
    */

    // 'Job()' is not given here so its job is associated with its parent.
    val context = Dispatchers.Default + CoroutineName("MyContext") 

    // Then 'launch' a coroutine using the created 'context'
    val job = launch(context) {
        // 'coroutineContext' has the reference to the 'context' of the coroutine
        println("Hello: \n `running on $coroutineContext`")
    }
    job.join()

    // Inherits its context from its outer scope, e.g. 'runBlocking'
    async {
        println("world: \n  `running on $coroutineContext`")
    }.await() // We can also 'join' the returned job like this 
    
    println("Program finished \n `running on $coroutineContext`")
}

main()

Program started 
 `running on [BlockingCoroutine{Active}@7d751ab8, BlockingEventLoop@6baa4da7]`
Hello: 
 `running on [CoroutineName(MyContext), StandaloneCoroutine{Active}@5a0c9a55, Dispatchers.Default]`
world: 
  `running on [DeferredCoroutine{Active}@1b464713, BlockingEventLoop@6baa4da7]`
Program finished 
 `running on [BlockingCoroutine{Active}@7d751ab8, BlockingEventLoop@6baa4da7]`


In [3]:
// Explicitly defining the 'Job' for the coroutine context and launching a coroutine with a given context

 
// To create a custom context, the structure is like [CoroutineDispatcher]+[Job]+[CoroutineName].
/* 
'contextJob' can be used to control the lifecyle of 
the launched corotuines within 'dedicatedJobContext'; there might be several coroutines
launched in this context.
*/
val contextJob = Job();
val dedicatedJobContext: CoroutineContext = Dispatchers.IO + contextJob + CoroutineName("dedicatedJobContext")
    
fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started")

    // Here, 'context' inherits the 'Job' from the outer scope, e.g. 'runBlocking'. 
    val context = Dispatchers.Default + CoroutineName("context")
    
    
    // Let's launch a coroutine using the created context.
    // 'job' lifcycle is tied to 'context' whose 'job' is tied to 'runBlocking'.
    val job = launch(context) {
        println("Hello: \n `running on $coroutineContext`")
    }
    job.join()

    // Then we launch a coroutine using the created context with dedicated job.
    // 'def' lifcycle is tied to 'dedicatedJobContext'.
    val def = async(dedicatedJobContext) {
        println("World: \n `running on $coroutineContext`")
    }
    def.await()

    // 'cancDef' lifcycle is tied to 'dedicatedJobContext'.
    val cancDef = async(dedicatedJobContext) {
        println("After Cancelling the dedicatedJobContext: \n `running on $coroutineContext`")
        delay(500)
        /*
        VERY INPORTANT: A delay is added here for consistency.
        We have added 'dedicatedJobContext.cancel()' at the end of the cell and
        cancelling is cooperative and there must be a suspension point for cancellation as will be shown
        in the next section. 
        */
        println("After a short delay: \n `running on $coroutineContext`")
    } // Won't run because its context's job is already cancelled. Awating for it will throw an exception.

    /*
    IMPORTANT: If somehow 'runBlocking' gets cancelled, it is not responsible of
    cancelling the coroutines launched in 'dedicatedJobContext' because it has its own 
    dedicated 'Job'.
    */
    
    println("Program finished")
}

main()

/* 
VERY INPORTANT: We are responsible for the context lifecycle management
because we have defined a dedicated 'job' for it;
we must 'cancel()' its 'Job' when we are done.
*/
dedicatedJobContext.cancel() 

// We can use '[Coroutine Context/Job].isActive' to check if they are active.
if (!dedicatedJobContext.isActive) println("The `dedicatedJobContext` is no longer active")

Program started
Hello: 
 `running on [CoroutineName(context), StandaloneCoroutine{Active}@478e9761, Dispatchers.Default]`
World: 
 `running on [CoroutineName(dedicatedJobContext), DeferredCoroutine{Active}@bbf52f7, Dispatchers.IO]`
Program finished
After Cancelling the dedicatedJobContext: 
 `running on [CoroutineName(dedicatedJobContext), DeferredCoroutine{Active}@7b6f7ba7, Dispatchers.IO]`
`dedicatedJobContext` is no longer active


## Using `withContext` to Dispatch Coroutines Execution

In [4]:
// Switching the dispatcher using 'withContext'

fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started")

    /*
    'withContext' returns a suspend function and simultaneously switches the context.
    At call point, the function gets suspended at caller context and swithces to the given context.
    */
    // Here, only dispatcher is given as the context.
    withContext(Dispatchers.IO) {
        // Runs on IO Threads pool
        println("Hello World")
    }
    
    println("Program finished")
}

/*
It is also good practice to define it as a suspend function:

suspend fun changeContext() = withContext(context) {
    println("Hello World")
}
*/

main()

Program started
Hello World
Program finished


In [5]:
// Switching the context using 'withContext'

fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started")

    /*
    'withContext' returns a suspend function and simultaneously switches the context.
    At call point, the function gets suspended at call point and switches to the given context.
    */
    val context = Dispatchers.IO + CoroutineName("MyContext")
    withContext(context) {
        // Runs on IO Threads pool
        println("Hello World")
    }
    
    println("Program finished")
}

main()

Program started
Hello World
Program finished


## `CoroutineContext` with `CoroutineScope`

In [6]:
// Assigning a customised context to a custom coroutine scope

// Let's create customized contexts.
val contextJob = Job()
val context: CoroutineContext = Dispatchers.IO + contextJob + CoroutineName("MyContext")

// We can directly define a scope and launch jobs in that scope.
val myScope = CoroutineScope(context)
    
fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started: \n `running on $coroutineContext`")
    
    // Inherits its context form the outer scope.
    coroutineScope {
        println("Hello World 0: \n `running on $coroutineContext`")
        // There is an option to use 'launch' with any given context.
        launch(context) { println("Hello World 1: \n `running on $coroutineContext`") }
    }

    // Now, the launch notations is [My Coroutine Scope].[launch{}/asnyc{}]
    // Let's launch in 'myScope' and join 
    myScope.launch { 
        println("Hello World 2: \n `running on $coroutineContext`")
    }.join()

    // Let's only launch in 'myScope' without joining.
    myScope.launch { 
        println("Hello World 3: \n `running on $coroutineContext`")
        delay(500)
        /*
        VERY INPORTANT: A delay is added here for consistency.
        We have added 'myScope.cancel()' at the end of the cell and
        cancelling is cooperative and there must be a suspension point for cancellation as will be shown
        in the next section. 
        */
        println("Hello World 4: \n `running on $coroutineContext`")
    } // Without 'join()'
    
    /*
    IMPORTANT: If somehow 'runBlocking' gets cancelled, it is not responsible of
    cancelling the coroutines launched in 'myScope' because it has its own 
    'context' with a dedicated 'Job'.
    */
    println("Program finished: \n `running on $coroutineContext`")
}

main()

/* 
VERY INPORTANT: We are responsible for the context lifecycle management
because we have defined a dedicated 'job' for it;
we must 'cancel()' its 'Job' when we are done.
*/
myScope.cancel() 

Program started: 
 `running on [BlockingCoroutine{Active}@6295ba6b, BlockingEventLoop@599001a7]`
Hello World 0: 
 `running on [ScopeCoroutine{Active}@11dcb8ae, BlockingEventLoop@599001a7]`
Hello World 1: 
 `running on [CoroutineName(MyContext), StandaloneCoroutine{Active}@c2366df, Dispatchers.IO]`
Hello World 2: 
 `running on [CoroutineName(MyContext), StandaloneCoroutine{Active}@31a5fc4, Dispatchers.IO]`
Hello World 3: 
 `running on [CoroutineName(MyContext), StandaloneCoroutine{Active}@dddb1d7, Dispatchers.IO]`
Program finished: 
 `running on [BlockingCoroutine{Active}@6295ba6b, BlockingEventLoop@599001a7]`


In [7]:
// Creating multi-threaded context

/* 
VERY IMPORTANT: Threads are resource-intensive, so it's important to create them judiciously. 
In most cases, it's better to rely on Kotlin's default dispatchers (Dispatchers.Default, Dispatchers.IO).
*/
// Let's create customized multi-thread contexts
val threadNo = 6
val contextJob = Job()
val contextDispatcher = Executors.newFixedThreadPool(threadNo).asCoroutineDispatcher()
val context: CoroutineContext = contextDispatcher + contextJob + CoroutineName("MyContext")
// We can directly define a scope and launch jobs in that scope
val myScope = CoroutineScope(context)
    
fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started: \n `running on $coroutineContext`")

    // Launch multiple jobs in this custom scope
    // We create a 'List' of 'Job's
    val jobs = List(10) { i ->
        myScope.launch {
            println("Job $i started on thread: ${Thread.currentThread().name}")
            delay(1000L)  // Simulating some work
            println("Job $i finished on thread: ${Thread.currentThread().name}")
        }
    }

    // Wait for all jobs to complete
    jobs.joinAll()

    println("Program finished: \n `running on $coroutineContext`")
}

main()

/* 
VERY INPORTANT: We are responsible for the context lifecycle management
because we have defined a dedicated 'job' for it;
we must 'cancel()' its 'Job' when we are done.
*/
myScope.cancel()

Program started: 
 `running on [BlockingCoroutine{Active}@59a69957, BlockingEventLoop@2522970f]`
Job 0 started on thread: pool-1-thread-1
Job 1 started on thread: pool-1-thread-2
Job 2 started on thread: pool-1-thread-3
Job 6 started on thread: pool-1-thread-2
Job 8 started on thread: pool-1-thread-2
Job 7 started on thread: pool-1-thread-1
Job 5 started on thread: pool-1-thread-6
Job 4 started on thread: pool-1-thread-5
Job 3 started on thread: pool-1-thread-4
Job 9 started on thread: pool-1-thread-1
Job 6 finished on thread: pool-1-thread-6
Job 1 finished on thread: pool-1-thread-3
Job 0 finished on thread: pool-1-thread-2
Job 2 finished on thread: pool-1-thread-5
Job 8 finished on thread: pool-1-thread-4
Job 7 finished on thread: pool-1-thread-1
Job 5 finished on thread: pool-1-thread-6
Job 4 finished on thread: pool-1-thread-3
Job 3 finished on thread: pool-1-thread-2
Job 9 finished on thread: pool-1-thread-5
Program finished: 
 `running on [BlockingCoroutine{Active}@59a69957, Bloc

## **DYE**: Do Your Experiment

In [ ]:
// Ready for your codes